In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import torch

In [3]:
from storage.har_datasets import HARTHDataset, UCI_HARDataset, sts_medoids, StreamingTimeSeriesCopy, split_by_test_subject
from s3ts.api.dms.har_datasets import LDFDataset, DFDataset
from storage.label_mappings import harth_label_mapping
from s3ts.api.nets.methods import create_model_from_DM, train_model, test_model
from torchvision.transforms import Normalize

In [4]:
ds = HARTHDataset("./datasets/HARTH/", wsize=48, normalize=True, label_mapping=harth_label_mapping)
print(len(ds))

6225494


In [5]:
if not os.path.exists("./datasets/HARTH/meds.npz"):
    meds = sts_medoids(ds, n=500)
    with open("./datasets/HARTH/meds.npz", "wb") as f:
        np.save(f, meds)
else:
    meds = np.load("./datasets/HARTH/meds.npz")

dfds = DFDataset(ds, patterns=meds, w=0.1, dm_transform=None, ram=False)

Loading cached dissimilarity frames if available...


In [6]:
DM = []

np.random.seed(42)
for i in np.random.choice(np.arange(len(dfds)), 500):
    dm, _, _ = dfds[i]
    DM.append(dm)

DM = torch.stack(DM)

dm_transform = Normalize(mean=DM.mean(dim=[0, 2, 3]), std=DM.std(dim=[0, 2, 3]))
dfds.dm_transform = dm_transform

In [9]:
data_split = split_by_test_subject(ds, 21)

dm = LDFDataset(dfds, data_split=data_split, batch_size=128, random_seed=42, num_workers=8, reduce_train_inbalance=True)

TypeError: 'bool' object is not callable

In [8]:
print(len(dm.ds_train))
print(len(dm.ds_val))
print(len(dm.ds_test))

6047162
178332
178332


In [8]:
model = create_model_from_DM(dm, name=None, 
        dsrc="img", arch="cnn", task="cls")

Input shape:  torch.Size([1, 12, 48, 48])
Latent shape:  torch.Size([1, 40, 3, 48])


In [ ]:
model, data = train_model(dm, model, max_epochs=2)
print(data)